# **Import Libraries**

In [ ]:
#Set path to MAIN FOLDER OF EXPERIMENT
#cd /path/to/EXPERIMENT_FOLDER/

In [18]:
#Import Models APIs
from tensorflow.keras.layers import Input, Average, Dropout, Dense, GlobalAveragePooling2D
from keras.layers import Resizing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.applications import inception_resnet_v2
from tensorflow.keras.applications import efficientnet 
from tensorflow.keras.applications import densenet
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.applications import nasnet
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras.applications import resnet_v2
from tensorflow.keras.applications import xception
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
import itertools
import tensorflow as tf
import pandas as pd
import numpy as np
import math
import os

print("LIBRARIES LOADED")

LIBRARIES LOADED


In [19]:
#Tweakable parameters
MODEL_KIND = "teacher_model"
MODEL_NAME = "EnsembleModel"

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 4

ENSEMBLE_FILES = ['EfficientNetB7',
                  'DenseNet201',
                  'DenseNet121',
                  'InceptionResNetV2', 
                  'InceptionV3', 
                  'NASNetLarge', 
                  'EfficientNetB0'] 
#                   'Xception', 
#                   'MobileNetV2', 
#                   'ResNet152V2', 
#                   'ResNet50V2', 
#                   'NASNetMobile']

#Data paths
MAIN_DATA_DIR = "ds/"
TRAIN_DATA_DIR = MAIN_DATA_DIR + "train/"
TEST_DATA_DIR = MAIN_DATA_DIR + "test/"
VALIDATION_DATA_DIR = MAIN_DATA_DIR + "val/"

#Model path
TEACHER_MODEL_PATH = 'models/teacher_model/'

print("ALL REQUERED PATHS SET")

ALL REQUERED PATHS SET


In [20]:
#Save Model Function
def save_m(model, directory, model_name):
    if not os.path.exists(directory):
        os.makedirs(directory)
    model.save(directory + "/" + model_name + ".h5")

#Load model Function
def load_m(directory, model_name):
    if not os.path.exists(directory):
        print("Model File Does Not Exist!!")
        return 
    model = load_model(directory + "/" + model_name + ".h5")
    return model

#Get Ensemble Function with given input shape
def get_ensemble(all_models, to_ensemble, input_shape=IMAGE_SIZE):
	emodels = []
	ensemble_input = Input(shape=(input_shape[0], input_shape[1], 3), name='ensemble_input')
	for model_name in to_ensemble:
		emodel = all_models[model_name]
		emodel = add_preprocess_layer(emodel, processing_layers[model_name], ensemble_input)
		emodel = rename_all_layers(emodel, model_name)
		emodels.append(emodel)

	outputs = [m.output for m in emodels]
	y = Average()(outputs) 
	model = Model(inputs=ensemble_input , outputs=y, name=MODEL_KIND + '-' + MODEL_NAME)
	return model

#Add resizing and preprocessing layer to model
def add_preprocess_layer(emodel, pre_layer, ensemble_input):
	input_shape = emodel.inputs[0].shape
	x = Resizing(input_shape[1], input_shape[2])(ensemble_input)
	x = pre_layer()(x)
	outputs = emodel(x)
	model = Model(inputs = ensemble_input, outputs = outputs)
	return model

#Rename all layers except input layer of model
def rename_all_layers(model, file):
	for i in range(1, len(model.layers)):
		model.layers[i]._name = model.layers[i]._name + '_' + file

	return model

print("ALL CUSTOM FUNCTIONS DEFIEND")

ALL CUSTOM FUNCTIONS DEFIEND


**Preprocessing Layers**

In [21]:
class EN_PreprocessLayer(tf.keras.layers.Layer):
    def __init__(self, name="preprocess", **kwargs):
        super(EN_PreprocessLayer, self).__init__(name=name, **kwargs)
        self.preprocess = efficientnet.preprocess_input

    def call(self, input):
        return self.preprocess(input)

    def get_config(self):
        config = super(EN_PreprocessLayer, self).get_config()
        return config

class Dense_PreprocessLayer(tf.keras.layers.Layer):
    def __init__(self, name="preprocess", **kwargs):
        super(Dense_PreprocessLayer, self).__init__(name=name, **kwargs)
        self.preprocess = densenet.preprocess_input

    def call(self, input):
        return self.preprocess(input)

    def get_config(self):
        config = super(Dense_PreprocessLayer, self).get_config()
        return config

class IRNV2_PreprocessLayer(tf.keras.layers.Layer):
    def __init__(self, name="preprocess", **kwargs):
        super(IRNV2_PreprocessLayer, self).__init__(name=name, **kwargs)
        self.preprocess = inception_resnet_v2.preprocess_input

    def call(self, input):
        return self.preprocess(input)

    def get_config(self):
        config = super(IRNV2_PreprocessLayer, self).get_config()
        return config

class INV3_PreprocessLayer(tf.keras.layers.Layer):
    def __init__(self, name="preprocess", **kwargs):
        super(INV3_PreprocessLayer, self).__init__(name=name, **kwargs)
        self.preprocess = inception_v3.preprocess_input

    def call(self, input):
        return self.preprocess(input)

    def get_config(self):
        config = super(INV3_PreprocessLayer, self).get_config()
        return config
    
class NN_PreprocessLayer(tf.keras.layers.Layer):
    def __init__(self, name="preprocess", **kwargs):
        super(NN_PreprocessLayer, self).__init__(name=name, **kwargs)
        self.preprocess = nasnet.preprocess_input

    def call(self, input):
        return self.preprocess(input)

    def get_config(self):
        config = super(NN_PreprocessLayer, self).get_config()
        return config

class XCT_PreprocessLayer(tf.keras.layers.Layer):
    def __init__(self, name="preprocess", **kwargs):
        super(XCT_PreprocessLayer, self).__init__(name=name, **kwargs)
        self.preprocess = xception.preprocess_input

    def call(self, input):
        return self.preprocess(input)

    def get_config(self):
        config = super(XCT_PreprocessLayer, self).get_config()
        return config

class MNV2_PreprocessLayer(tf.keras.layers.Layer):
    def __init__(self, name="preprocess", **kwargs):
        super(MNV2_PreprocessLayer, self).__init__(name=name, **kwargs)
        self.preprocess = mobilenet_v2.preprocess_input

    def call(self, input):
        return self.preprocess(input)

    def get_config(self):
        config = super(MNV2_PreprocessLayer, self).get_config()
        return config

class RNV2_PreprocessLayer(tf.keras.layers.Layer):
    def __init__(self, name="preprocess", **kwargs):
        super(RNV2_PreprocessLayer, self).__init__(name=name, **kwargs)
        self.preprocess = resnet_v2.preprocess_input

    def call(self, input):
        return self.preprocess(input)

    def get_config(self):
        config = super(RNV2_PreprocessLayer, self).get_config()
        return config

processing_layers = {'EfficientNetB7':EN_PreprocessLayer, 
                     'DenseNet121':Dense_PreprocessLayer,
                     'DenseNet201':Dense_PreprocessLayer,
                     'InceptionResNetV2':IRNV2_PreprocessLayer,
                     'InceptionV3':INV3_PreprocessLayer, 
                     'NASNetLarge':NN_PreprocessLayer, 
                     'EfficientNetB0':EN_PreprocessLayer, 
                     'Xception':XCT_PreprocessLayer, 
                     'MobileNetV2':MNV2_PreprocessLayer, 
                     'ResNet152V2':RNV2_PreprocessLayer, 
                     'ResNet50V2':RNV2_PreprocessLayer, 
                     'NASNetMobile':NN_PreprocessLayer }

print("ALL REQUERED PREPROCESSING LAYERS DEFINED")

ALL REQUERED PREPROCESSING LAYERS DEFINED


In [12]:
teacher_models = {}

for file in ENSEMBLE_FILES:
	teacher_models[file] = {}

#Actual models
teacher_models['DenseNet201'] = load_m(TEACHER_MODEL_PATH + 'DenseNet201', 'DenseNet201')
teacher_models['DenseNet121'] = load_m(TEACHER_MODEL_PATH + 'DenseNet121', 'DenseNet121')
teacher_models['InceptionResNetV2'] = load_m(TEACHER_MODEL_PATH + 'InceptionResNetV2', 'InceptionResNetV2')
teacher_models['InceptionV3'] = load_m(TEACHER_MODEL_PATH + 'InceptionV3', 'InceptionV3')
teacher_models['EfficientNetB7'] = load_m(TEACHER_MODEL_PATH + 'EfficientNetB7', 'EfficientNetB7')
teacher_models['NASNetLarge'] = load_m(TEACHER_MODEL_PATH + 'NASNetLarge', 'NASNetLarge')
teacher_models['EfficientNetB0'] = load_m(TEACHER_MODEL_PATH + 'EfficientNetB0', 'EfficientNetB0')
teacher_models['Xception'] = load_m(TEACHER_MODEL_PATH + 'Xception', 'Xception')
teacher_models['MobileNetV2'] = load_m(TEACHER_MODEL_PATH + 'MobileNetV2', 'MobileNetV2')
teacher_models['ResNet152V2'] = load_m(TEACHER_MODEL_PATH + 'ResNet152V2', 'ResNet152V2')
teacher_models['ResNet50V2'] = load_m(TEACHER_MODEL_PATH + 'ResNet50V2', 'ResNet50V2')
teacher_models['NASNetMobile'] = load_m(TEACHER_MODEL_PATH + 'NASNetMobile', 'NASNetMobile')

print("[INFO] Load models")

[INFO] Load models


In [13]:
#LOAD TEST & VAL DATA
val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

if not os.path.exists(VALIDATION_DATA_DIR):
    print("VALIDATION DATA DOES NOT EXITS!")
else:
    print("LOAD VALIDATION SAMPLES...")
    validation_generator = val_datagen.flow_from_directory(
        VALIDATION_DATA_DIR,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        seed=42,
        shuffle=False)

    #CHECK  THE NUMBER OF SAMPLES
    nb_validation_samples = len(validation_generator.filenames)
    if nb_validation_samples == 0:
        print("NO DATA VALIDATION FOUND IN VALIDATION FOLDER!")

print()
if not os.path.exists(TEST_DATA_DIR):
    print("TEST DATA DOES NOT EXITS!")
else:
    print("LOAD TEST SAMPLES...")
    test_generator = test_datagen.flow_from_directory(
                TEST_DATA_DIR,
                target_size=IMAGE_SIZE,
                batch_size=BATCH_SIZE,
                class_mode='categorical',
                seed=42,
                shuffle=False)

    #CHECK  THE NUMBER OF SAMPLES
    nb_test_samples = len(test_generator.filenames)
    if nb_test_samples == 0:
        print("NO DATA TEST FOUND IN TEST FOLDER!")

LOAD VALIDATION SAMPLES...
Found 1194 images belonging to 199 classes.

LOAD TEST SAMPLES...
Found 796 images belonging to 199 classes.


# **ENSEMBLE MODEL ABLATIONS**

In [23]:
TOTAL_CLASSES = 199
LEARNING_RATE = 0.001
OPTIMIZER = Adam
BATCH_SIZE = 4
EPOCHS = 30
DROPOUT_RATE = 0.5

print("HYPERPARAMETERS")
print("---------------------")
print("BATCH_SIZE -->", BATCH_SIZE)
print("EPOCHS SET -->", EPOCHS)
print("DROPOUT_RATE -->", DROPOUT_RATE)
print("LEARNING_RATE -->", LEARNING_RATE)
print("OPTIMIZER -->", OPTIMIZER.__name__)

HYPERPARAMETERS
---------------------
BATCH_SIZE --> 4
EPOCHS SET --> 30
DROPOUT_RATE --> 0.5
LEARNING_RATE --> 0.001
OPTIMIZER --> Adam


In [26]:
import itertools
import time

n = 2
i = 1
max_val_acc = 0
best_ablation = None

start_time = time.time()

print("The following models are to be ensembled")
print()
print(ENSEMBLE_FILES)

while(n <= len(ENSEMBLE_FILES)):
    for to_ensemble in list(itertools.combinations(ENSEMBLE_FILES, n)):
        print()
        print("Current Ablation number", str(i) + " :", to_ensemble)
        model = get_ensemble(teacher_models, to_ensemble)
        model.compile(
            optimizer=Adam(learning_rate=LEARNING_RATE),
            loss = CategoricalCrossentropy(from_logits=True),
            metrics=['accuracy']
        )
        acc = model.evaluate(validation_generator)[1]
        if max_val_acc < acc:
            max_val_acc = acc
            best_ablation = to_ensemble

        i += 1
    n += 1

elapsed_time = time.time() - start_time
train_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

print()
print()
print(train_time, 'elapsed time')
print()
print(elapsed_time, 'seconds')
print()
print("Best Ablation:", best_ablation)
print("Highest Accuracy:", max_val_acc)
print("Total ablations built:", n)

The following models are to be ensembled

['EfficientNetB7', 'DenseNet201', 'DenseNet121', 'InceptionResNetV2', 'InceptionV3', 'NASNetLarge', 'EfficientNetB0']

Current Ablation number 1 : ('EfficientNetB7', 'DenseNet201')
299/299 [==============================] - 34s 90ms/step - loss: 4.4121 - accuracy: 0.9246

Current Ablation number 2 : ('EfficientNetB7', 'DenseNet121')
299/299 [==============================] - 31s 80ms/step - loss: 4.4326 - accuracy: 0.9079

Current Ablation number 3 : ('EfficientNetB7', 'InceptionResNetV2')
299/299 [==============================] - 38s 99ms/step - loss: 4.4215 - accuracy: 0.9154

Current Ablation number 4 : ('EfficientNetB7', 'InceptionV3')
299/299 [==============================] - 30s 78ms/step - loss: 4.4260 - accuracy: 0.9179

Current Ablation number 5 : ('EfficientNetB7', 'NASNetLarge')
299/299 [==============================] - 44s 115ms/step - loss: 4.4273 - accuracy: 0.9137

Current Ablation number 6 : ('EfficientNetB7', 'EfficientNetB0

299/299 [==============================] - 90s 211ms/step - loss: 4.4263 - accuracy: 0.9322

Current Ablation number 94 : ('EfficientNetB7', 'DenseNet201', 'DenseNet121', 'InceptionResNetV2', 'EfficientNetB0')
299/299 [==============================] - 71s 176ms/step - loss: 4.4337 - accuracy: 0.9255

Current Ablation number 95 : ('EfficientNetB7', 'DenseNet201', 'DenseNet121', 'InceptionV3', 'NASNetLarge')
299/299 [==============================] - 81s 189ms/step - loss: 4.4281 - accuracy: 0.9380

Current Ablation number 96 : ('EfficientNetB7', 'DenseNet201', 'DenseNet121', 'InceptionV3', 'EfficientNetB0')
299/299 [==============================] - 59s 148ms/step - loss: 4.4355 - accuracy: 0.9271

Current Ablation number 97 : ('EfficientNetB7', 'DenseNet201', 'DenseNet121', 'NASNetLarge', 'EfficientNetB0')
299/299 [==============================] - 77s 186ms/step - loss: 4.4360 - accuracy: 0.9296

Current Ablation number 98 : ('EfficientNetB7', 'DenseNet201', 'InceptionResNetV2', 'Inc

In [27]:
#EVALUATION OF BEST ABLATION
teacher_ensemble = get_ensemble(teacher_models, best_ablation)
teacher_ensemble.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss = CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

teacher_ensemble.summary()

Model: "teacher_model-EnsembleModel"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ensemble_input (InputLayer)     [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
tf.image.resize_449_EfficientNe (None, 224, 224, 3)  0           ensemble_input[0][0]             
__________________________________________________________________________________________________
tf.image.resize_450_DenseNet201 (None, 224, 224, 3)  0           ensemble_input[0][0]             
__________________________________________________________________________________________________
tf.image.resize_451_DenseNet121 (None, 224, 224, 3)  0           ensemble_input[0][0]             
________________________________________________________________________

In [28]:
print("Evaluation on Validation Set")
acc = teacher_ensemble.evaluate(validation_generator)

print("Evaluation on Test Set")
acc = teacher_ensemble.evaluate(test_generator)

Evaluation on Validation Set
299/299 [==============================] - 86s 201ms/step - loss: 4.4281 - accuracy: 0.9380
Evaluation on Test Set
199/199 [==============================] - 40s 200ms/step - loss: 4.4158 - accuracy: 0.9259


In [29]:
#save model
save_m(teacher_ensemble, TEACHER_MODEL_PATH + MODEL_NAME, MODEL_NAME)
print("[INFO] MODEL AND HISTORY SAVED to ", TEACHER_MODEL_PATH + 'TeacherEnsemble/')

c:\users\franc\anaconda3\envs\replicate\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


[INFO] MODEL AND HISTORY SAVED to  models/teacher_model/TeacherEnsemble/
